In [1]:
import sys
import joblib
import numpy as np
import pandas as pd

In [2]:
sys.path.append('../archive/src')
#sys.path

In [3]:
from predictor import ScoringService

In [4]:
inputs = ScoringService.get_inputs("../input/")

In [5]:
dfs = ScoringService.get_dataset(inputs)

Mem. usage decreased to 217.60 Mb (43.8% reduction)
Mem. usage decreased to  0.29 Mb (21.9% reduction)
Mem. usage decreased to 20.13 Mb (28.3% reduction)
Mem. usage decreased to 306.26 Mb (44.1% reduction)


In [6]:
api_dfs = joblib.load("../tmp/dfs.joblib")

In [7]:
def adapt_dfs(dfs, api_dfs):
    from pandas.api.types import is_numeric_dtype
    sort_map = {
        "stock_labels": ["Local Code", "base_date"],
        "stock_list": ["Effective Date"],
        "stock_fin": ["Local Code", "base_date"],
        "stock_price": ["Local Code", "EndOfDayQuote Date"],
    }
    out_dfs = {}
    
    for k, df in dfs.items():
        cols = df.columns
        out_df = api_dfs[k][cols].copy()

        for col in cols:
            dtype = df[col].dtype
            if is_numeric_dtype(dtype):
                if dtype == "bool":
                    out_df[col] = out_df[col] == "True"
                else:
                    out_df[col] = pd.to_numeric(out_df[col]).astype(dtype)
            elif (dtype == "O"):
                #print(col)
                out_df[col] = out_df[col].astype(dtype)
                out_df[col] = out_df[col].mask(out_df[col] == "")
            else:
                out_df[col] = out_df[col].astype(dtype)
        if k in sort_map:
            out_df = out_df.sort_values(sort_map[k])
        out_dfs[k] = out_df
    return out_dfs


dfs = adapt_dfs(dfs, api_dfs)

In [8]:
ls_df = dfs["stock_list"]
dfs["stock_list"] = ls_df[ls_df["Effective Date"] < "2021-03-01"]
dfs["stock_list"].tail()

,prediction_target,Effective Date,Local Code,Name (English),Section/Products,33 Sector(Code),33 Sector(name),17 Sector(Code),17 Sector(name),Size Code (New Index Series),Size (New Index Series),IssuedShareEquityQuote AccountingStandard,IssuedShareEquityQuote ModifyDate,IssuedShareEquityQuote IssuedShare
430,True,2021-02-26,3758,Aeria Inc.,JASDAQ(Standard / Domestic),5250,Information & Communication,10,"IT & SERVICES, OTHERS",-,-,ConsolidatedJP,2021/02/12,23649428.0
1544,True,2021-02-26,4650,"SD ENTERTAINMENT,Inc.",JASDAQ(Standard / Domestic),9050,Services,10,"IT & SERVICES, OTHERS",-,-,ConsolidatedJP,2021/02/10,8997000.0
517,True,2021-02-26,7975,"LIHIT LAB.,INC.",Second Section(Domestic),3800,Other Products,10,"IT & SERVICES, OTHERS",-,-,ConsolidatedJP,2021/01/08,1907850.0
406,True,2021-02-26,2001,NIPPN CORPORATION,First Section (Domestic),3050,Foods,1,FOODS,6,TOPIX Small 1,ConsolidatedJP,2021/02/04,78824008.0
1138,True,2021-02-26,2681,GEO HOLDINGS CORPORATION,First Section (Domestic),6100,Retail Trade,14,RETAIL TRADE,6,TOPIX Small 1,ConsolidatedJP,2021/02/08,42405952.0


In [9]:
tb_df = ScoringService.get_table(dfs)
tb_df.shape

(84769, 183)

In [10]:
tb_df.last_flg.mean()

0.8509714636246741

In [11]:
tb_df.tail()

,base_date,Local Code,label_date_5,label_high_5,label_low_5,label_date_10,label_high_10,label_low_10,label_date_20,label_high_20,...,feat_pbr,feat_roe,feat_ma_roe,feat_fc_roe,feat_dv_fill_Result_Dividend AnnualDividendPerShare,feat_amv_ratio_trading_value,feat_amv_ratio_Result_FinancialStatement NetAssets,feat_amv_ratio_Result_FinancialStatement TotalAssets,feat_dprfill_Result_Dividend AnnualDividendPerShare,ID
254222,2020-07-31,9997,2020-08-07,0.268311,0.097473,2020-08-17,0.268311,0.097473,2020-08-31,0.453857,...,7.017102e+05,0.014374,0.015032,-0.000289,0.021362,0.003538,1.425090e-06,0.000003,1.042836e+06,2020-07-31-9997
254225,2020-10-29,9997,2020-11-06,0.011124,-0.177979,2020-11-13,0.011124,-0.177979,2020-11-30,0.011124,...,9.265572e+05,0.014374,0.015032,0.014080,0.016178,0.003895,1.079264e-06,0.000002,1.042836e+06,2020-10-29-9997
254228,2020-10-30,9997,2020-11-09,0.078918,-0.042389,2020-11-16,0.078918,-0.042389,2020-12-01,0.169556,...,7.826461e+05,0.013565,0.016843,0.022514,0.018846,0.020330,1.277717e-06,0.000003,1.087290e+06,2020-10-30-9997
254231,2021-01-28,9997,2021-02-04,0.023163,-0.128662,2021-02-12,0.023163,-0.128662,2021-03-01,0.023163,...,1.074871e+06,0.013565,0.016843,0.024884,0.013722,0.002830,9.303443e-07,0.000002,1.087290e+06,2021-01-28-9997
254234,2021-01-29,9997,2021-02-05,-0.015656,-0.116516,2021-02-15,-0.015656,-0.116516,2021-03-02,-0.015656,...,1.017155e+06,0.051817,0.022484,0.023876,0.013913,0.002822,9.831340e-07,0.000002,2.731107e+05,2021-01-29-9997


In [12]:
tb_df.columns.tolist()

['base_date',
 'Local Code',
 'label_date_5',
 'label_high_5',
 'label_low_5',
 'label_date_10',
 'label_high_10',
 'label_low_10',
 'label_date_20',
 'label_high_20',
 'label_low_20',
 'past_label_date_5',
 'feat_past_label_high_5',
 'feat_past_label_low_5',
 'past_label_date_10',
 'feat_past_label_high_10',
 'feat_past_label_low_10',
 'past_label_date_20',
 'feat_past_label_high_20',
 'feat_past_label_low_20',
 'Result_FinancialStatement AccountingStandard',
 'Result_FinancialStatement FiscalPeriodEnd',
 'Result_FinancialStatement ReportType',
 'Result_FinancialStatement FiscalYear',
 'Result_FinancialStatement ModifyDate',
 'Result_FinancialStatement CompanyType',
 'Result_FinancialStatement ChangeOfFiscalYearEnd',
 'Result_FinancialStatement NetSales',
 'Result_FinancialStatement OperatingIncome',
 'Result_FinancialStatement OrdinaryIncome',
 'Result_FinancialStatement NetIncome',
 'Result_FinancialStatement TotalAssets',
 'Result_FinancialStatement NetAssets',
 'Result_FinancialSt

In [13]:
n_trg_names = tb_df["Local Code"].nunique()
n_trg_names

3524

In [14]:
tb_df.base_date.max()

Timestamp('2021-03-26 00:00:00')

In [15]:
org_size = tb_df.shape[0]
tb_df.dropna(subset=["label_high_20", "label_low_20"], inplace=True)
tb_df = tb_df[tb_df.base_date.dt.year > 2016]
valid_size = tb_df.shape[0]
org_size, valid_size, org_size - valid_size

(84769, 68562, 16207)

In [16]:
tb_df.base_date.max()

Timestamp('2021-02-26 00:00:00')

In [17]:
y = tb_df[["label_high_20", "label_low_20"]].values
y.shape

(68562, 2)

In [18]:
y

array([[ 0.1566 ,  0.00401],
       [ 0.1359 ,  0.00177],
       [ 0.0444 , -0.02632],
       ...,
       [ 0.1696 , -0.0424 ],
       [ 0.02316, -0.1364 ],
       [-0.01566, -0.1243 ]], dtype=float16)

In [19]:

# w = tb_df.w.values
# w.shape

In [20]:
tb_df.memory_usage(deep=True).sort_values().sum()

165528618

In [21]:
from preprocessor import Preprocessor
transformer = Preprocessor()
X = transformer.fit_transform(tb_df)
X.shape

(68562, 66)

In [22]:
X.head()

,,,feat_past_label_high_5,feat_past_label_low_5,feat_past_label_high_10,feat_past_label_low_10,feat_past_label_high_20,feat_past_label_low_20,feat_ratio_Result_FinancialStatement TotalAssets,feat_ratio_Result_FinancialStatement NetAssets,feat_ratio_diff_Result_FinancialStatement NetSales,feat_ratio_diff_Result_FinancialStatement OperatingIncome,...,feat_amv_ratio_Result_FinancialStatement TotalAssets,feat_dprfill_Result_Dividend AnnualDividendPerShare,feat_cat_Result_FinancialStatement AccountingStandard,feat_cat_Result_FinancialStatement ReportType,feat_cat_Result_FinancialStatement CompanyType,feat_cat_Forecast_FinancialStatement ReportType,feat_cat_Section/Products,feat_cat_33 Sector(Code),feat_cat_17 Sector(Code),feat_cat_Size Code (New Index Series)
ID,base_date,Local Code,,,,,,,,,,,,,,,,,,,,,
2017-02-10-1301,2017-02-10,1301,0.011032,-0.000740,0.008430,-0.004398,0.010292,-0.015060,0.024527,0.100867,0.180698,1.068504,...,0.000004,4.326320e+04,ConsolidatedJP,Q3,GB,Annual,First Section (Domestic),50,1,7
2017-02-17-1301,2017-02-17,1301,0.034973,0.004009,0.044495,-0.000740,0.037994,-0.009132,0.024527,0.100867,0.180698,1.068504,...,0.000004,4.326320e+04,ConsolidatedJP,Q3,GB,Annual,First Section (Domestic),50,1,7
2017-05-11-1301,2017-05-11,1301,0.035767,-0.001360,0.041443,-0.003771,0.038971,-0.024948,0.029416,0.100845,0.185395,-26.029412,...,0.000003,-2.428507e+07,ConsolidatedJP,Annual,GB,Annual,First Section (Domestic),50,1,7
2017-08-04-1301,2017-08-04,1301,0.033600,-0.011200,0.033600,-0.012802,0.036926,-0.020874,0.056970,0.111546,0.088840,1.096360,...,0.000003,8.696246e+05,ConsolidatedJP,Q1,GB,Annual,First Section (Domestic),50,1,7
2017-11-06-1301,2017-11-06,1301,0.083313,-0.004169,0.061218,-0.036743,0.086365,-0.022278,0.124369,0.131017,0.108953,0.799716,...,0.000003,7.459579e+05,ConsolidatedJP,Q2,GB,Annual,First Section (Domestic),50,1,7


In [23]:
X.columns.tolist()

['feat_past_label_high_5',
 'feat_past_label_low_5',
 'feat_past_label_high_10',
 'feat_past_label_low_10',
 'feat_past_label_high_20',
 'feat_past_label_low_20',
 'feat_ratio_Result_FinancialStatement TotalAssets',
 'feat_ratio_Result_FinancialStatement NetAssets',
 'feat_ratio_diff_Result_FinancialStatement NetSales',
 'feat_ratio_diff_Result_FinancialStatement OperatingIncome',
 'feat_ratio_diff_Result_FinancialStatement OrdinaryIncome',
 'feat_ratio_diff_Result_FinancialStatement NetIncome',
 'feat_ratio_ma_Result_FinancialStatement NetSales',
 'feat_ratio_ma_Result_FinancialStatement OperatingIncome',
 'feat_ratio_ma_Result_FinancialStatement OrdinaryIncome',
 'feat_ratio_ma_Result_FinancialStatement NetIncome',
 'feat_ratio_fill_Result_FinancialStatement CashFlowsFromOperatingActivities',
 'feat_ratio_fill_Result_FinancialStatement CashFlowsFromFinancingActivities',
 'feat_ratio_fill_Result_FinancialStatement CashFlowsFromInvestingActivities',
 'feat_ratio_Result_FinancialStateme

In [24]:
X.dtypes.index[X.dtypes == "category"]

Index(['feat_cat_Result_FinancialStatement AccountingStandard',
       'feat_cat_Result_FinancialStatement ReportType',
       'feat_cat_Result_FinancialStatement CompanyType',
       'feat_cat_Forecast_FinancialStatement ReportType',
       'feat_cat_Section/Products', 'feat_cat_33 Sector(Code)',
       'feat_cat_17 Sector(Code)', 'feat_cat_Size Code (New Index Series)'],
      dtype='object')

In [25]:
tb_df.quarter_span.value_counts()

3.0     68442
6.0        53
9.0        15
2.0        12
4.0        10
7.0         5
5.0         4
12.0        2
1.0         1
8.0         1
Name: quarter_span, dtype: int64

In [26]:
na_ratio = X.isna().mean()
na_ratio[na_ratio>0.5]

Series([], dtype: float64)

In [27]:
joblib.dump(
    {"X": X, "y": y, "n_trg_names": n_trg_names, "transformer":transformer},
    "../tmp/preprocessed.joblib",
    compress=False
)

['../tmp/preprocessed.joblib']